<a href="https://colab.research.google.com/github/hannah-rae/gee_tutorials/blob/main/Remote_Sensing_for_Sustainable_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Remote Sensing for Sustainable Development
# Satellite Data Analysis in Google Earth Engine
Google Earth Engine (GEE) is a cloud-based platform for interacting with and analyzing petabytes of satellite and other Earth data sets. GEE can be used with the Javascript browser-based code editor (https://code.earthengine.google.com) or the Python API. The main benefit of GEE is that it allows you to access huge remote sensing data sets and perform analysis entirely on Google's infrastructure without having to download files to install libraries on your own computer... for free!

This tutorial shows how to use the GEE Python API to load example remote sensing data sets and perform basic analyses. We will learn how to:
1.   define a region of interest (ROI) (`ee.Geometry`)
2.   load satellite data sets (`ee.ImageCollection`)
3.   filter data sets by time and regions of interest
4.   compute band indices (e.g., NDWI)
5.   analyze trends using linear regression
6.   perform clustering analysis with K-means to reveal different land cover classes

To run this Colab notebook, you will need a Google Earth Engine account (https://signup.earthengine.google.com/#!/) and a Google Drive account.


## Set up your environment

Install the Google Earth Engine API

In [ ]:
!pip install earthengine-api

Authenticate your Google Earth Engine account

In [ ]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=7BNazRURUt5PZB-DDi34PEJ-dzK-IBSNvJsJZ6F2jR4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiLFBf7tLV_rsaQmNd76yzCEHnbidyw4HJ0DsKK8GfQtOzm-unRUBQ

Successfully saved authorization token.


Import the Earth Engine API and initialize it.

In [ ]:
import ee
ee.Initialize()

Install `geemap`, a python library that provides useful functions for the GEE Python API (https://github.com/giswqs/geemap)

NOTE: You may get an error in this step that says, "You must restart the runtime in order to use newly installed versions." This is a known issue - you will need to click the "Restart Runtime" button and re-run the steps from the beginning (i.e., you will need to authenticate twice).

In [ ]:
!pip install geemap

Import `geemap`. We import the `eefolium` version because the default version uses `ipyleaflet`, which is not supported in Colab yet.

In [ ]:
import geemap.eefolium as geemap

## Displaying the map

GEE enables you to visualize your data and outputs on a map using `folium` or other python libraries for map visualization. The `geemap` library provides useful wrapper functions for visualizing the map in just a couple of lines.

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map # Display the Map object

## Define a region of interest (ROI)

http://bboxfinder.com can be used to draw a bounding box and get the coordinates for that bounding box anywhere in the world.

Here is a bounding box around Lake Mead, Nevada, USA:
http://bboxfinder.com/#35.934305,-115.214535,36.441939,-114.045762

Copy the min/max longitude/latitude from bboxfinder:

In [ ]:
xmin,ymin,xmax,ymax = -115.214535,35.934305,-114.045762,36.441939

Create an `ee.Geometry.Rectangle` object defined by those coordinates.

In [ ]:
bbox = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

Then we add our bounding box to the map and give it the name Lake Mead.

In [ ]:
Map = geemap.Map()
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10
Map.addLayerControl() # Allows us to control which layers we are viewing on the map
Map

## Load satellite data sets

The Google Earth Engine catalog documents the vast archive of remote sensing data sets that you can access through GEE: 
https://developers.google.com/earth-engine/datasets

One of the most frequently used satellite data sets is from the Landsat-8 satellite. Landsat acquires multispectral (visible, infrared, thermal) images at 30m/pixel resolution with 16 day revisit time:
https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR 

Data sets like Landsat can be loaded as `ee.ImageCollection` objects. An `ImageCollection` contains many `ee.Image` objects from various times, regions, etc. 

In [ ]:
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

## Filter data sets

Now, our landsat variable contains this entire Landsat-8 archive. We want to filter this archive to only contain images for our specific ROI. This is done with the `filterBounds()` function.

In [ ]:
landsat = landsat.filterBounds(bbox)

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10
Map.addLayer(landsat, name='Landsat8') # Add the Landsat-8 collection filtered by our bbox
Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

...why is it all gray?!

This is because Landsat-8 has many bands with different ranges of values, and we haven't told it how to visualize the data. The GEE Catalog documents what these bands are: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR#bands

This catalog tells us that the red, green, and blue bands are `'B4'`, `'B3'`, and `'B2'`. It also gives us a code example of good visualization parameters for this data set. We'll use those.

In [ ]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10
Map.addLayer(landsat, visParams, name='Landsat8') # Add the Landsat-8 collection filtered by our bbox
Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

Much better! 

Now, we want to filter our data set by a time period that we are interested in. Let's use 2019. We can filter by start and end dates using the filterDate() function.

In [ ]:
landsat = landsat.filterDate('2019-01-01', '2019-12-31')

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10
Map.addLayer(landsat, visParams, name='Landsat8') # Add the Landsat-8 collection filtered by our bbox
Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

Our map looks a bit different now, since we are displaying only images from 2019. By default, GEE visualizes the most recent observations on top (like a stack). Since the winter months are more cloudy, we see lots of clouds in the images on the top of the stack. 

We could use a cloud mask to remove the clouds, but for now, let's just restrict the time period even further to the summer.

In [ ]:
landsat = landsat.filterDate('2019-05-01', '2019-08-31')

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10
Map.addLayer(landsat, visParams, name='Landsat8') # Add the Landsat-8 collection filtered by our bbox
Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

Now we are pretty much cloud-free. (But notice the seam lines!)

Here we are still displaying a collection of all the images that intersected with our bounding box, all stacked on top of each other. To fuse them into one image, we can use the `mosaic()` function. This stitches the images into one, keeping the most recently acquired images on top. 

In [ ]:
landsat_mosaic = landsat.mosaic()

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10
Map.addLayer(landsat_mosaic, visParams, name='Landsat8') # Add the Landsat-8 collection filtered by our bbox
Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

You can also use other functions to composite the images, such as `min()` or `max()` or `median()`. We can add a different composite to the map to see how it compares to the last-on-top method.

In [ ]:
landsat_med = landsat.median()
landsat_min = landsat.min()
landsat_max = landsat.max()

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(landsat_mosaic, visParams, name='last on top') 
Map.addLayer(landsat_med, visParams, name='median') 
Map.addLayer(landsat_min, visParams, name='minimum')
Map.addLayer(landsat_max, visParams, name='maximum') 

Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

What can we notice about the difference between these compositing methods? 

What are the different artifacts we see in each? 

Which looks like the best?

Now we've created a single cloud-free image from our collection of images. Since this was created by stitching together all the images that intersected with our bounding box, our image is still much larger than our bounding box. To clip an image to an ROI, we can use the `clip()` function.

In [ ]:
landsat_mosaic = landsat_mosaic.clip(bbox)

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(landsat_mosaic, visParams, name='last on top') 

Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

Landsat-8 observations have 30m/pixel spatial resolution and acquire repeat images every 16 days. 

Sentinel-2 is another frequently-used Earth observation satellite that has 10m resolution and 5-day revisit frequency. While Landsat is managed by NASA/USGS, Sentinel-2 is managed by ESA. We can access this data set in GEE too:
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR')

We used multiple functions to progressively filter the Landsat data set. We can also chain these all together in one line.

In [ ]:
s2 = s2.filterBounds(bbox).filterDate('2019-05-01', '2019-8-31').mosaic().clip(bbox)

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(landsat_mosaic, visParams, name='Landsat-8') # Add the Landsat layer
Map.addLayer(s2, visParams, name='Sentinel-2')  # Add the Sentinel-2 layer

Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

What differences can we see comparing the images at these two resolutions?

## Compute band indices

Ratios between different spectral bands, called "band indices," are commonly used to emphasize or reveal different land cover patterns in satellite imagery. 

For example, NDWI (GREEN-NIR/GREEN+NIR) is used to emphasize the presence of water. High NDWI corresponds with water bodies and low NDWI corresponds with vegetation; moderate values usually correspond with built-up areas (e.g., cities or roads).

In [ ]:
ndwi = s2.normalizedDifference(['B3', 'B8'])

NDWI values range from -1 to 1. We can define new visualization parameters to display NDWI. 

In [ ]:
ndwi_vis = {
    'min': -1,
    'max':1,
    'palette': ['964B00', 'FFFFFF', '3498DB']
}

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(s2, visParams, name='Sentinel-2')  # Add the Sentinel-2 layer
Map.addLayer(ndwi, ndwi_vis, name='NDWI')  # Add the NDWI layer

Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

What do we notice? Which objects have high NDWI? Low NDWI?

## Linear regression

Linear regression can be used to quantify broad trends over time. Linear regression is simply fitting a line ($y=mx+b$) to your data such that your choice of m and b minimize the distance between data samples and your line.

Here, we'll use linear regression to quantify how the water level around Lake Mead (via NDWI) is changing over time. Since the Landsat-8 record goes back to 2013 and the Sentinel-2 record only goes back to ~2017, we'll use Landsat-8 so we can see longer-term trends. 

In this setup, our independent ($x$) variable is time and our dependent ($y$) variable is NDWI. 

First, let's see what sort of changes we should expect  to see by visualizing NDWI in 2013 compared to 2020.

In [ ]:
# Get median NDWI for our ROI in 2013
l8_ndwi_2013 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
               .filterDate('2013-05-01', '2013-08-31')\
               .filterBounds(bbox)\
               .median()\
               .normalizedDifference(['B3', 'B5'])\
               .clip(bbox)

In [ ]:
# Get median NDWI for our ROI in 2020
l8_ndwi_2020 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
               .filterDate('2020-05-01', '2020-08-31')\
               .filterBounds(bbox)\
               .median()\
               .normalizedDifference(['B3', 'B5'])\
               .clip(bbox)

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(landsat_med, visParams, name='Landsat')  # Add the Landsat-8 layer
Map.addLayer(l8_ndwi_2013, ndwi_vis, name='Landsat NDWI 2013')  # Add the 2013 NDWI layer
Map.addLayer(l8_ndwi_2020, ndwi_vis, name='Landsat NDWI 2020')  # Add the 2020 NDWI layer

Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

What kind of changes do we see between 2013 and 2020? 

We want to analyze trends across the whole Landsat-8 image collection record, from 2013-present, in our ROI. So we filter the collection by our ROI and this time period.

In [ ]:
def createTimeBand(image):
  # Scale milliseconds by a large constant to avoid very small slopes
  # in the linear regression output.
  return image.addBands(image.metadata('system:time_start').divide(1e18))

# Add NDWI band
def addNDWIBand(image):
  ndwi = image.normalizedDifference(['B3', 'B5']).rename('ndwi')
  return image.addBands(ndwi)

# Load the input image collection: projected climate data.
l8_coll = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
               .filterDate('2013-01-01', '2020-12-31')\
               .filterBounds(bbox)

Now, for linear regression, we need to set up our independent (x, time) and dependent (y, NDWI) variables. We'll add these two variables as bands to each image in the collection. We can do this by creating helper functions `createTimeBand()` and `addNDWIBand()` and applying them to every image in the collection using the `map()` function.

In [ ]:
l8_coll = l8_coll.map(createTimeBand) # Add the time band to each image in the collection
l8_coll = l8_coll.map(addNDWIBand) # Add NDWI band to each image in the collection

We can use the GEE function `ee.Reducer.linearFit()` to use linear regression to fit a line to each pixel in our data. We select the time and NDWI bands that we added to the images in our collection to be the x and y variables.

This function returns an image with two bands: 'scale' (the slope) and 'offset' (the y-intercept).

In [ ]:
# Reduce the collection with the linear fit reducer.
# Independent variable are followed by dependent variables.
linearFit = l8_coll.select(['system:time_start', 'ndwi'])\
                      .reduce(ee.Reducer.linearFit())

We can visualize trends using the slope of the best-fit line: negative values show decreasing water and increasing values show water gain. We'll visualize losses as red and gains as green.

In [ ]:
lr_vis = {
    'bands': 'scale',
    'min': -1200000,
    'max': 1200000,
    'palette': ['FF0000', 'FFFFFF', '00FF00']
}

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(landsat_med, visParams, name='Landsat')  # Add the Landsat-8 layer
Map.addLayer(l8_ndwi_2013, ndwi_vis, name='Landsat NDWI 2013')  # Add the 2013 NDWI layer
Map.addLayer(l8_ndwi_2020, ndwi_vis, name='Landsat NDWI 2020')  # Add the 2020 NDWI layer
Map.addLayer(linearFit.clip(bbox), lr_vis, name='NDWI trend')  # Visualize the linear regression slope in each pixel

Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

Where do we see NDWI is increasing or decreasing?

## Cluster analysis

**K-means** is a popular unsupervised clustering algorithm that tries to assign data points to *k* clusters such that each sample belongs to the cluster whose mean is closest to the sample value. 

The standard algorithm begins by randomly initializing $k$ cluster centroids and iteratively performing two steps: 
1. assigning points to the centroid with the nearest cluster mean, and 
2. re-computing each cluster mean (i.e., the mean of all samples assigned to the cluster). 

The algorithm converges and stops updating cluster assignments when the cluster assignment is no longer changing with each update. The Euclidean distance is typically used as the distance metric for finding the nearest centroid. 

The number of clusters *k* is a tunable hyperparameter, which could be chosen based on domain knowledge (e.g., number of land cover classes) or such that within-cluster variance is minimized.

GEE provides the `ee.Clusterer.wekaKMeans()` object for doing K-means clustering. (Weka is a machine learning library that this implementation is based off of, hence the name `wekaKMeans`.)
https://developers.google.com/earth-engine/guides/clustering

First, we create a training data set by randomly sampling pixels in our NDWI data set.

In [ ]:
# Make the training dataset.
training = ndwi.sample(**{
  'region': bbox, # restrict the sampling to our bounding box
  'scale': 30, # sample within 30m/pixel cells, i.e. the data set resolution
  'numPixels': 1000 # number of samples to draw
})

Instantiate a clusterer object and specify how many clusters to use, then train it using the training points.

In [ ]:
clusterer = ee.Clusterer.wekaKMeans(3).train(training)

Cluster (predict on) the data set using the trained clusterer model.

In [ ]:
result = ndwi.cluster(clusterer)

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='Lake Mead') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(s2, visParams, name='Sentinel-2')  # Add the Sentinel-2 layer
Map.addLayer(ndwi, ndwi_vis, name='NDWI')  # Add the NDWI layer
Map.addLayer(result.randomVisualizer(), name='Clusters') # Add the clustered result with random colors

Map.addLayerControl() # Add layer control to toggle layers
Map # Display the map

What patterns do we see? 

What happens if we change the number of clusters? 

# Customize your analysis

Now that you've gone through these examples, you can try doing this analysis for another region you're interested in. Try using http://bboxfinder.com/ to draw your own bounding box and replace the `bbox` coordinates with your own.